Refining data cleaning and processing skills to create a AAPL volatility index based on the calculations of the VIX

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
# loading options data
options_df = pd.read_csv("AAPL.csv")
# cleaning
options_df[options_df["bid price"] >= 0]

# correct expiration dates for 30 day VIX
T = (pd.to_datetime(options_df["expiration date"]) - pd.to_datetime(options_df["date"])).dt.days
options_df["days to expiration"] = T

# creating moneyness, spread columns
options_df["moneyness"] = options_df["closing price"] - options_df["strike price"]
options_df["spread"] = round(options_df["ask price"] - options_df["bid price"], 3) 

# creating mid price
options_df["mid price"] = (options_df["ask price"] + options_df["bid price"])/2

# concatenating call and put rows for the same price    
options_df["call mid"] = 0
options_df["put mid"] = 0
options_df["call spread"] = 0
options_df["put spread"] = 0
    
# boolean masks for call and put options in every second row
call_mask = (options_df.iloc[::2, 2] == "call").values
put_mask = (options_df.iloc[::2, 2] == "put").values

# indices of every second row
indices = np.arange(0, options_df.shape[0], 2)

# assign values based on call options
options_df.iloc[indices[call_mask], 19] = options_df.iloc[indices[call_mask], 18].values
options_df.iloc[indices[call_mask], 20] = options_df.iloc[indices[call_mask] + 1, 18].values
options_df.iloc[indices[call_mask], 21] = options_df.iloc[indices[call_mask], 17].values
options_df.iloc[indices[call_mask], 22] = options_df.iloc[indices[call_mask] + 1, 17].values

# assign values based on put options
options_df.iloc[indices[put_mask], 20] = options_df.iloc[indices[put_mask], 18].values
options_df.iloc[indices[put_mask], 19] = options_df.iloc[indices[put_mask] + 1, 18].values
options_df.iloc[indices[put_mask], 22] = options_df.iloc[indices[put_mask], 17].values
options_df.iloc[indices[put_mask], 21] = options_df.iloc[indices[put_mask] + 1, 17].values

# days to expiration greater than 20 and less than 33 
mask1 = options_df["days to expiration"] > 20
mask2 = options_df["days to expiration"] < 33
options_df = options_df[mask1]
options_df = options_df[mask2]

options_df.head(2)

C:\Users\hollanderj\AppData\Local\Temp\ipykernel_22096\2519819793.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  options_df = options_df[mask2]


,date,expiration date,type,strike price,ask price,ask size,bid price,bid size,last price,volume,...,date div,exp date div,days to expiration,moneyness,spread,mid price,call mid,put mid,call spread,put spread
464,2016-01-04,2016-01-29,put,17.5,0.0300,1356.0,0.0025,2176.0,0.0325,80.0,...,0.085246,0.123607,25,8.837,0.028,0.01625,8.825,0.01625,0.175,0.028
465,2016-01-04,2016-01-29,call,17.5,8.9125,76.0,8.7375,92.0,8.2000,8.0,...,0.085246,0.123607,25,8.837,0.175,8.82500,0.000,0.00000,0.000,0.000


In [ ]:
# reducing rows to every other
options_df = options_df[options_df.index % 2 == 0]
# rearranging index
options_df.index = np.arange(0, len(options_df))
options_df.head()
options_df = options_df.drop(["bid price", "ask price", "last price",
"volume", "open interest", "exp closing price", "date div", "exp date div","type"], axis=1)
options_df.to_csv("AAPL_by_strike", index=True)

In [ ]:
# creating new dataframe for date organization
df = pd.DataFrame()

# converting index to date column
if "date" in df.columns:
    df = df.reset_index()

# organizing based on dates
for header in ["date", "expiration date", "days to expiration", "closing price", "moneyness"
, "spread", "call mid", "put mid", "call spread", "put spread"]:
    df[header] = options_df.groupby("date")[header].apply(list)

df.head(2)

,date,expiration date,days to expiration,closing price,moneyness,spread,call mid,put mid,call spread,put spread
date,,,,,,,,,,
2016-01-04,"[2016-01-04, 2016-01-04, 2016-01-04, 2016-01-0...","[2016-01-29, 2016-01-29, 2016-01-29, 2016-01-2...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[26.337, 26.337, 26.337, 26.337, 26.337, 26.33...","[8.837, 7.587, 6.337, 5.087, 3.836999999999999...","[0.028, 0.01, 0.023, 0.025, 0.175, 0.018, 0.01...","[8.825, 7.60625, 6.375, 5.1375, 3.9375, 2.8, 1...","[0.01625, 0.0325, 0.058750000000000004, 0.0875...","[0.175, 0.162, 0.15, 0.15, 0.175, 0.15, 0.05, ...","[0.028, 0.01, 0.023, 0.025, 0.013, 0.018, 0.01..."
2016-01-05,"[2016-01-05, 2016-01-05, 2016-01-05, 2016-01-0...","[2016-01-29, 2016-01-29, 2016-01-29, 2016-01-2...","[24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 2...","[25.677, 25.677, 25.677, 25.677, 25.677, 25.67...","[8.177, 6.927, 5.677, 4.427, 3.176999999999999...","[0.015, 0.008, 0.01, 0.015, 0.01, 0.02, 0.02, ...","[8.21875, 6.975, 5.75625, 4.5375, 3.36875, 2.2...","[0.0175, 0.02875, 0.052500000000000005, 0.095,...","[0.087, 0.075, 0.087, 0.075, 0.062, 0.062, 0.0...","[0.015, 0.008, 0.01, 0.015, 0.01, 0.02, 0.02, ..."


In [ ]:
for i in range(20):
    print(sorted(set(df.iloc[i, 2])))

[25, 32]
[24, 31]
[23, 30]
[22, 29]
[21, 28]
[25, 32]
[24, 31]
[23, 30]
[22, 29]
[21, 28]
[24, 31]
[23, 30]
[22, 29]
[21, 28]
[25, 32]
[24, 31]
[23, 30]
[22, 29]
[21, 28]
[25, 32]


In [ ]:
# importing rates data
mo1 = pd.read_csv("1Month_2016.csv")
mo3 = pd.read_csv("3Month_2016.csv")
mo6 = pd.read_csv("6Month_2016.csv")
yr1 = pd.read_csv("1Year_2016.csv")

# combining data and dropping duplicate dates
rates = pd.concat([mo1, mo3, mo6, yr1], axis=1)
# df.columns.duplicated returns True for duplicated columns (for columns [a, b, a]: [False, False, True])
# ~ inverts ([True, True, False]) so it only keeps non duplicates 
rates = rates.loc[:,~rates.columns.duplicated()]

#  renaming columns and filling na 
rates = rates.rename(columns={"DGS1MO":"1MO","DGS3MO":"3MO","DGS6MO":"6MO","DGS1":"1YR"})
rates = rates.replace(".", np.NaN).fillna(method="ffill")
rates[rates["DATE"] == "2016-01-18"]

,DATE,1MO,3MO,6MO,1YR
10,2016-01-18,0.19,0.24,0.37,0.49


In [ ]:
#